In [50]:
import gpxpy.geo as geo
import gpxpy.gpx

In [29]:
with open('26kLongRun.gpx', 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)


In [30]:
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            print(f'Point at ({point.latitude},{point.longitude}) -> {point.elevation}')

Point at (59.317436,18.050744) -> 22.0
Point at (59.317436,18.050744) -> 22.0
Point at (59.317442,18.050786) -> 22.0
Point at (59.317375,18.050818) -> 22.0
Point at (59.317321,18.050844) -> 21.0
Point at (59.31728,18.050833) -> 21.0
Point at (59.317276,18.050802) -> 21.0
Point at (59.316755,18.051057) -> 21.0
Point at (59.316744,18.050972) -> 21.0
Point at (59.316671,18.051002) -> 21.0
Point at (59.316655,18.05086) -> 21.0
Point at (59.316546,18.050918) -> 21.0
Point at (59.316422,18.050833) -> 21.0
Point at (59.316269,18.050892) -> 21.0
Point at (59.316202,18.050867) -> 21.0
Point at (59.315165,18.051376) -> 19.0
Point at (59.315091,18.051467) -> 19.0
Point at (59.314918,18.051618) -> 19.0
Point at (59.314849,18.051655) -> 18.0
Point at (59.314849,18.051655) -> 18.0
Point at (59.314918,18.051618) -> 19.0
Point at (59.314751,18.050461) -> 17.0
Point at (59.314645,18.049544) -> 19.0
Point at (59.314562,18.047639) -> 16.0
Point at (59.314581,18.045036) -> 16.0
Point at (59.314493,18.0442

In [31]:
for waypoint in gpx.waypoints:
    print('waypoint {0} -> ({1},{2})'.format(waypoint.name, waypoint.latitude, waypoint.longitude))

In [32]:
for route in gpx.routes:
    print('Route:')
    for point in route.points:
        print('Point at ({0},{1}) -> {2}'.format(point.latitude, point.longitude, point.elevation))

In [33]:
gpx.add_missing_speeds()
moving_data = gpx.get_moving_data(raw=True)
print('GPX:', gpx.waypoints, gpx.routes, gpx.tracks)

GPX: [] [] [GPXTrack(name='26k Long Run', segments=[GPXTrackSegment(points=[...])])]


In [68]:
import pandas as pd
df = pd.DataFrame(gpx.get_points_data())
df.head()

,point,distance_from_start,track_no,segment_no,point_no
0,"[trkpt:59.317436,18.050744@22.0@2024-03-31 00:...",0.000000,0,0,0
1,"[trkpt:59.317436,18.050744@22.0@2024-03-31 00:...",0.000000,0,0,1
2,"[trkpt:59.317442,18.050786@22.0@2024-03-31 00:...",2.477509,0,0,2
3,"[trkpt:59.317375,18.050818@22.0@2024-03-31 00:...",10.154226,0,0,3
4,"[trkpt:59.317321,18.050844@21.0@2024-03-31 00:...",16.424508,0,0,4


In [70]:
df.point


0      [trkpt:59.317436,18.050744@22.0@2024-03-31 00:...
1      [trkpt:59.317436,18.050744@22.0@2024-03-31 00:...
2      [trkpt:59.317442,18.050786@22.0@2024-03-31 00:...
3      [trkpt:59.317375,18.050818@22.0@2024-03-31 00:...
4      [trkpt:59.317321,18.050844@21.0@2024-03-31 00:...
                             ...                        
726    [trkpt:59.313742,18.038573@8.0@2024-03-31 02:3...
727    [trkpt:59.313926,18.039625@11.0@2024-03-31 02:...
728    [trkpt:59.31445,18.044056@16.0@2024-03-31 02:3...
729    [trkpt:59.314512,18.044145@15.0@2024-03-31 02:...
730    [trkpt:59.314534,18.044098@15.0@2024-03-31 02:...
Name: point, Length: 731, dtype: object

In [73]:
for i in range(len(df.point)):
    print(df.point[i].latitude, df.point[i].longitude, df.point[i].elevation, df.point[i].time, df.point[i].speed)


59.317436 18.050744 22.0 2024-03-31 00:00:00+00:00 None
59.317436 18.050744 22.0 2024-03-31 00:00:00+00:00 None
59.317442 18.050786 22.0 2024-03-31 00:00:01+00:00 None
59.317375 18.050818 22.0 2024-03-31 00:00:04+00:00 None
59.317321 18.050844 21.0 2024-03-31 00:00:06+00:00 None
59.31728 18.050833 21.0 2024-03-31 00:00:08+00:00 None
59.317276 18.050802 21.0 2024-03-31 00:00:08+00:00 None
59.316755 18.051057 21.0 2024-03-31 00:00:30+00:00 None
59.316744 18.050972 21.0 2024-03-31 00:00:32+00:00 None
59.316671 18.051002 21.0 2024-03-31 00:00:34+00:00 None
59.316655 18.05086 21.0 2024-03-31 00:00:37+00:00 None
59.316546 18.050918 21.0 2024-03-31 00:00:42+00:00 None
59.316422 18.050833 21.0 2024-03-31 00:00:47+00:00 None
59.316269 18.050892 21.0 2024-03-31 00:00:54+00:00 None
59.316202 18.050867 21.0 2024-03-31 00:00:56+00:00 None
59.315165 18.051376 19.0 2024-03-31 00:01:39+00:00 None
59.315091 18.051467 19.0 2024-03-31 00:01:43+00:00 None
59.314918 18.051618 19.0 2024-03-31 00:01:50+00:00

Moving data: MovingData(moving_time=9494.0, stopped_time=3628.0, moving_distance=26373.67664451617, stopped_distance=75.38365575734497, max_speed=5.301893465063272)


[GPXTrack(name='26k Long Run', segments=[GPXTrackSegment(points=[...])])]

Length: 26509.50496435586


15.828712396714733

1.0413623939143994

AttributeError: 'PointData' object has no attribute 'longitude'